In [9]:
from os import listdir
import os
from os.path import isfile, join
import numpy as np
import pandas as pd
import librosa
import json
from tqdm import tqdm
import torch
import csv

In [82]:
mypath = 'data/raw/Progressive_Rock_Songs/'
progressive_rock = [[os.path.join(dirpath,f), 1] for (dirpath, dirnames, filenames) in os.walk(mypath) for f in filenames if f.endswith('.mp3')]

In [83]:
mypath = 'data/raw/Not_Progressive_Rock/'
not_progressive_rock = [[os.path.join(dirpath,f), 0] for (dirpath, dirnames, filenames) in os.walk(mypath) for f in filenames if f.endswith('.mp3')]

In [84]:
dataset = not_progressive_rock + progressive_rock

In [86]:
my_df = pd.DataFrame(dataset)
my_df.to_csv('annotations_v2.csv', index=False, header=False)

In [87]:
pd.read_csv('annotations_v2.csv', header=None)

,0,1
0,data/raw/Not_Progressive_Rock/Other_Songs\01 -...,0
1,data/raw/Not_Progressive_Rock/Other_Songs\01 -...,0
2,data/raw/Not_Progressive_Rock/Other_Songs\01 -...,0
3,data/raw/Not_Progressive_Rock/Other_Songs\01 -...,0
4,data/raw/Not_Progressive_Rock/Other_Songs\01 -...,0
...,...,...
267,data/raw/Progressive_Rock_Songs/L'evoluzione.mp3,1
268,data/raw/Progressive_Rock_Songs/MIKE OLDFIELD ...,1
269,data/raw/Progressive_Rock_Songs/Pain of Salvat...,1
270,data/raw/Progressive_Rock_Songs/The Flower Kin...,1


In [35]:
relative_path = 'data/'
mypath = relative_path + 'raw/Progressive_Rock_Songs/'
progressive_rock = [[os.path.join(dirpath,f), 1] for (dirpath, dirnames, filenames) in os.walk(mypath) for f in filenames if f.endswith('.mp3')]
mypath = relative_path + 'raw/Not_Progressive_Rock/'
not_progressive_rock = [[os.path.join(dirpath,f), 0] for (dirpath, dirnames, filenames) in os.walk(mypath) for f in filenames if f.endswith('.mp3')]
dataset = not_progressive_rock + progressive_rock

In [40]:
from utils.audio import log_mel_spectrogram
os.makedirs('data/process/',exist_ok=True)

data = []
for i, (path, value) in enumerate(dataset):
    array = log_mel_spectrogram(path)
    new_path = f'data/process/tensor_{i:04d}.pt'
    torch.save(array, new_path)
    data += [[new_path, value]]

In [43]:
dataset = []
for i in range(int(len(data)/2)):
    dataset+=[[data[2*i],data[2*i+1]]]

In [45]:
my_df = pd.DataFrame(dataset)
my_df.to_csv('annotations_v3.csv', index=False, header=False)

In [6]:
relative_path = 'data/'
mypath = relative_path + 'cap6610fa22_test_set/Progressive_Rock_Songs/'
progressive_rock = [[os.path.join(dirpath,f), 1] for (dirpath, dirnames, filenames) in os.walk(mypath) for f in filenames if f.endswith('.mp3')]
mypath = relative_path + 'cap6610fa22_test_set/Not_Progressive_Rock/'
not_progressive_rock = [[os.path.join(dirpath,f), 0] for (dirpath, dirnames, filenames) in os.walk(mypath) for f in filenames if f.endswith('.mp3')]
dataset = not_progressive_rock + progressive_rock

In [7]:
len(dataset)

267

In [10]:
from utils.audio import log_mel_spectrogram
os.makedirs('data/process_test/',exist_ok=True)

data = []
for i, (path, value) in enumerate(dataset):
    array = log_mel_spectrogram(path)
    new_path = f'data/process_test/tensor_{i:04d}.pt'
    torch.save(array, new_path)
    data += [[new_path, value]]

In [12]:
data

[['data/process_test/tensor_0000.pt', 0],
 ['data/process_test/tensor_0001.pt', 0],
 ['data/process_test/tensor_0002.pt', 0],
 ['data/process_test/tensor_0003.pt', 0],
 ['data/process_test/tensor_0004.pt', 0],
 ['data/process_test/tensor_0005.pt', 0],
 ['data/process_test/tensor_0006.pt', 0],
 ['data/process_test/tensor_0007.pt', 0],
 ['data/process_test/tensor_0008.pt', 0],
 ['data/process_test/tensor_0009.pt', 0],
 ['data/process_test/tensor_0010.pt', 0],
 ['data/process_test/tensor_0011.pt', 0],
 ['data/process_test/tensor_0012.pt', 0],
 ['data/process_test/tensor_0013.pt', 0],
 ['data/process_test/tensor_0014.pt', 0],
 ['data/process_test/tensor_0015.pt', 0],
 ['data/process_test/tensor_0016.pt', 0],
 ['data/process_test/tensor_0017.pt', 0],
 ['data/process_test/tensor_0018.pt', 0],
 ['data/process_test/tensor_0019.pt', 0],
 ['data/process_test/tensor_0020.pt', 0],
 ['data/process_test/tensor_0021.pt', 0],
 ['data/process_test/tensor_0022.pt', 0],
 ['data/process_test/tensor_0023.p

In [13]:
my_df = pd.DataFrame(data)
my_df.to_csv('annotations_test_v3.csv', index=False, header=False)

In [8]:
mypath = 'data/cap6610fa22_test_set/Progressive Rock Songs/'
progressive_rock = [[os.path.join(dirpath,f), 1] for (dirpath, dirnames, filenames) in os.walk(mypath) for f in filenames if f.endswith('.mp3')]

In [9]:
mypath = 'data/cap6610fa22_test_set/Not_Progressive_Rock/'
not_progressive_rock = [[os.path.join(dirpath,f), 0] for (dirpath, dirnames, filenames) in os.walk(mypath) for f in filenames if f.endswith('.mp3')]

In [10]:
dataset = not_progressive_rock + progressive_rock

In [11]:
my_df = pd.DataFrame(dataset)
my_df.to_csv('annotations_test.csv', index=False, header=False)

In [14]:
import whisper

In [15]:
model = whisper.load_model("base.en")

C:\Users\trdalm\PycharmProjects\Project2\venv\lib\site-packages\whisper\__init__.py:44: UserWarning: C:\Users\trdalm\.cache\whisper\base.en.pt exists, but the SHA256 checksum does not match; re-downloading the file
  warnings.warn(f"{download_target} exists, but the SHA256 checksum does not match; re-downloading the file")
100%|███████████████████████████████████████| 139M/139M [00:48<00:00, 2.97MiB/s]


In [16]:
model.dims

ModelDimensions(n_mels=80, n_audio_ctx=1500, n_audio_state=512, n_audio_head=8, n_audio_layer=6, n_vocab=51864, n_text_ctx=448, n_text_state=512, n_text_head=8, n_text_layer=6)

In [17]:
model.encoder

AudioEncoder(
  (conv1): Conv1d(80, 512, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv2): Conv1d(512, 512, kernel_size=(3,), stride=(2,), padding=(1,))
  (blocks): ModuleList(
    (0): ResidualAttentionBlock(
      (attn): MultiHeadAttention(
        (query): Linear(in_features=512, out_features=512, bias=True)
        (key): Linear(in_features=512, out_features=512, bias=False)
        (value): Linear(in_features=512, out_features=512, bias=True)
        (out): Linear(in_features=512, out_features=512, bias=True)
      )
      (attn_ln): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (mlp): Sequential(
        (0): Linear(in_features=512, out_features=2048, bias=True)
        (1): GELU(approximate='none')
        (2): Linear(in_features=2048, out_features=512, bias=True)
      )
      (mlp_ln): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    )
    (1): ResidualAttentionBlock(
      (attn): MultiHeadAttention(
        (query): Linear(in_features=512, out_f

In [21]:
from utils.model import AudioClassifier, ModelDimensions

In [23]:
dims = ModelDimensions(n_mels=80, n_audio_ctx=1500, n_audio_state=512, n_audio_head=8, n_audio_layer=6)

In [24]:
model_audio = AudioClassifier(dims)

In [25]:
model_audio.encoder = model.encoder

In [26]:
torch.save(model_audio.state_dict(), 'models/pretrained_small.pth')